In [107]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

class WarehouseOptimization:
    def __init__(self, file_path):
        self.file_path = file_path
        self.model = None
        self.num_warehouses = 0
        self.num_clients = 0
        self.warehouse_data = []
        self.client_demands = []
        self.cost_df = None
        self.cost_arr = None
        self.x = None  # Warehouse decision variables
        self.y = None  # Allocation decision variables
        self.read_data()
        self.build_model()

    def read_data(self):
        """Read data from input file."""
        with open(self.file_path, 'r') as file:
            # Read all lines while ignoring empty lines and comments
            data_lines = [line.strip() for line in file if line.strip() and not line.startswith('#')]

            # Get number of warehouses and clients
            self.num_warehouses, self.num_clients = map(int, data_lines[0].split())

            # Warehouse data (capacity and fixed costs)
            for i in range(self.num_warehouses):
                capacity, fixed_cost = map(float, data_lines[i + 1].split())
                self.warehouse_data.append([capacity, fixed_cost])

            # Client demands
            for j in range(self.num_clients):
                self.client_demands.append(float(data_lines[self.num_warehouses + 1 + 4 * j]))

            # Cost data
            cost_data = []
            for j in range(self.num_clients):
                start = self.num_warehouses + 2 + 4 * j
                cost_values = [list(map(float, data_lines[start + k].split())) for k in range(3)]
                cost_data.append([item for sublist in cost_values for item in sublist])

            # Convert cost_data to DataFrame
            self.cost_df = pd.DataFrame(
                cost_data,
                columns=[f'Warehouse {i+1}' for i in range(self.num_warehouses)],
                index=[f'Client {j+1}' for j in range(self.num_clients)]
            )
        self.cost_arr = self.cost_df.to_numpy()

    def build_model(self):
        """Build the optimization model."""
        self.model = gp.Model("Warehouse_Optimization")

        # Decision variables
        self.x = self.model.addVars(self.num_warehouses, vtype=GRB.BINARY, name="x")
        self.y = self.model.addVars(self.num_clients, self.num_warehouses, vtype=GRB.CONTINUOUS, name="y")

        # Objective: Minimize total cost (fixed + transportation costs)
        self.model.setObjective(
            gp.quicksum(self.warehouse_data[j][1] * self.x[j] for j in range(self.num_warehouses)) +
            gp.quicksum(self.cost_arr[i][j]/self.client_demands[i] * self.y[i, j] for i in range(self.num_clients) for j in range(self.num_warehouses)),
            GRB.MINIMIZE
        )

        # Constraints
        # 1. Demand satisfaction: Each client's total demand must be fully met
        for i in range(self.num_clients):
            self.model.addConstr(
                gp.quicksum(self.y[i, j] for j in range(self.num_warehouses)) == self.client_demands[i],
                name=f"Demand_{i}"
            )

        # 2. Warehouse capacity: Allocated demand cannot exceed warehouse capacity
        for j in range(self.num_warehouses):
            self.model.addConstr(
                gp.quicksum(self.y[i, j] for i in range(self.num_clients)) <= self.warehouse_data[j][0] * self.x[j],
                name=f"Capacity_{j}"
            )

        # 3. No allocation to closed warehouses
        for i in range(self.num_clients):
            for j in range(self.num_warehouses):
                self.model.addConstr(
                    self.y[i, j] <= self.warehouse_data[j][0] * self.x[j],
                    name=f"No_Allocation_Closed_{i}_{j}"
                )

    def optimize(self):
        """Solve the optimization model."""
        self.model.optimize()

        if self.model.status == GRB.OPTIMAL:
            print("Optimal total cost: ", self.model.objVal)
            for j in range(self.num_warehouses):
                if self.x[j].x > 0.5:  # Warehouse is open
                    print(f"Warehouse {j+1} is open")
                    for i in range(self.num_clients):
                        if self.y[i, j].x > 0:
                            print(f"  Client {i+1} -> Warehouse {j+1}: {self.y[i, j].x}")
        else:
            print("No optimal solution found")

if __name__ == "__main__":
    file_path = 'instance.txt'  # Replace with your actual file path
    warehouse_optimization = WarehouseOptimization(file_path)
    warehouse_optimization.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 866 rows, 816 columns and 3216 nonzeros
Model fingerprint: 0xd3dcd698
Variable types: 800 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 1e+04]
Found heuristic solution: objective 2142157.4500
Presolve removed 80 rows and 1 columns
Presolve time: 0.00s
Presolved: 786 rows, 815 columns, 3055 nonzeros
Variable types: 800 continuous, 15 integer (15 binary)

Root relaxation: objective 1.040444e+06, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 